Homework3 (problem 2 Bayesian Information Criterion)

In [1]:
import numpy as np
from scipy.stats import multivariate_normal

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Calculate the BIC for the data

In [3]:
def calculate_bic(log_likelihood, k, N):
    """Implements the Bayesian Information Criterion (BIC) for model selection."""
    return -2 * log_likelihood + k * np.log(N)

Setting

In [4]:
# --- 0. Experiment Setup ---
N = 150  # Number of Data Points
d = 2    # Data Dimension
np.random.seed(42) # Set seed for reproducibility

# --- 0a. Generate Data ---
# You can choose the model from which to generate data.
# example: true_model_type = 'spherical', 'diagonal', or 'full'
true_model_type = 'full' # 'full' covariance structure
# true_model_type = 'spherical' # 'spherical' covariance structure
# true_model_type = 'diagonal' # 'diagonal' covariance structure

print(f"--- Generate data (N={N}, d={d}) ---")
print(f"True Generating Model Type: {true_model_type}")
if true_model_type == 'spherical': # Spherical Covariance Structure (Equal variance in all dimensions)
    true_mean = np.array([1, 2])
    true_sigma_sq = 1.5
    true_cov = true_sigma_sq * np.identity(d)
elif true_model_type == 'diagonal': # Diagonal Covariance Structure (Different variance in each dimension)
    true_mean = np.array([1, 2])
    true_cov = np.diag([1.0, 2.5]) # Diagonal covariance matrix
else: # 'full'
    true_mean = np.array([1, 2])
    true_cov = np.array([[1.0, 0.7],  # Covariance between x1 and x2
                         [0.7, 2.0]])

data = np.random.multivariate_normal(true_mean, true_cov, N)

print(f"Generated Data Shape: {data.shape}")
print(f"First 3 data points:\n{data[:3]}\n")

--- Generate data (N=150, d=2) ---
True Generating Model Type: full
Generated Data Shape: (150, 2)
First 3 data points:
[[1.25084825 2.72913226]
 [2.53850146 2.32733658]
 [0.66887439 1.76583271]]



In [5]:
# --- 1. Each model's common mean vector MLE ---
mu_hat_mle = np.mean(data, axis=0)
print(f"Predicted Mean Vector from Data (MLE): {mu_hat_mle}\n")


# --- 2. Each model's covariance estimation, parameter count, log-likelihood, and BIC calculation ---
results = []

# --- Model M1: Spherical covariance model ---
# Parameter: mu (d), sigma^2 (1) => k1 = d + 1
k_m1 = d + 1
# Sigma_m1 = sigma^2 * I
# MLE for sigma^2: (1/Nd) * sum(||x_i - mu_hat||^2)
# or sample covariance matrix's trace average
# S_full_mle = (1/N) * sum((x_i - mu_hat)(x_i - mu_hat)^T)
residuals_m1 = data - mu_hat_mle
S_full_mle = (residuals_m1.T @ residuals_m1) / N # (d x d) matrix

sigma_sq_hat_m1 = np.trace(S_full_mle) / d
Sigma_hat_m1 = sigma_sq_hat_m1 * np.identity(d)

log_likelihood_m1 = multivariate_normal.logpdf(data, mean=mu_hat_mle, cov=Sigma_hat_m1, allow_singular=False).sum()
bic_m1 = calculate_bic(log_likelihood_m1, k_m1, N)
results.append({'model': 'M1 (Spherical)', 'k': k_m1, 'log_likelihood': log_likelihood_m1, 'bic': bic_m1, 'Sigma_hat': Sigma_hat_m1})

# --- Model M2: Diagonal Covariance Model ---
# Parameter: mu (d), sigma_j^2 (d) => k2 = d + d = 2d
k_m2 = 2 * d
# Sigma_m2 = diag(sigma_1^2, ..., sigma_d^2)
# MLE for sigma_j^2: S_full_mle's diagonal elements
Sigma_hat_m2 = np.diag(np.diag(S_full_mle))

# if diagonal covariance matrix has very small variances, it may cause singular matrix issues
# possible to add a small value (epsilon) to the diagonal for numerical stability
# if np.any(np.diag(Sigma_hat_m2) < 1e-6):
# Sigma_hat_m2 += np.finfo(float).eps * np.identity(d)


log_likelihood_m2 = multivariate_normal.logpdf(data, mean=mu_hat_mle, cov=Sigma_hat_m2, allow_singular=False).sum()
bic_m2 = calculate_bic(log_likelihood_m2, k_m2, N)
results.append({'model': 'M2 (Diagonal)', 'k': k_m2, 'log_likelihood': log_likelihood_m2, 'bic': bic_m2, 'Sigma_hat': Sigma_hat_m2})

# --- Model M3: Full Covariance Model ---
# Parameter: mu (d), Sigma (d*(d+1)/2) => k3 = d + d*(d+1)/2
k_m3 = d + d * (d + 1) // 2
# Sigma_m3 = full covariance matrix
# MLE for Sigma: S_full_mle (it is already calculated above)
Sigma_hat_m3 = S_full_mle

log_likelihood_m3 = multivariate_normal.logpdf(data, mean=mu_hat_mle, cov=Sigma_hat_m3, allow_singular=False).sum()
bic_m3 = calculate_bic(log_likelihood_m3, k_m3, N)
results.append({'model': 'M3 (Full)', 'k': k_m3, 'log_likelihood': log_likelihood_m3, 'bic': bic_m3, 'Sigma_hat': Sigma_hat_m3})

# --- 3. Printing Results And Selecting Model ---
print("--- Results from Each Model ---")
for res in results:
    print(f"Model: {res['model']}")
    print(f"  Number of Parameters (k): {res['k']}")
    print(f"  Predicted Covariance Matrix (Sigma_hat):\n{res['Sigma_hat']}")
    print(f"  Maximum Log Likelihood: {res['log_likelihood']:.4f}")
    print(f"  BIC score: {res['bic']:.4f}\n")

# choose the model with the lowest BIC
best_model = min(results, key=lambda x: x['bic'])

print("--- Final Model Selection (Lowest BIC) ---")
print(f"Selected Model: {best_model['model']}")
print(f"Selected Model's BIC: {best_model['bic']:.4f}")

# For futher experiments, you can change the true_model_type and run the code again.
# (e.g., true_model_type = 'spherical' or 'diagonal')
# You can also observe how BIC changes by varying the amount of data (N).

Predicted Mean Vector from Data (MLE): [0.99238766 1.94506753]

--- Results from Each Model ---
Model: M1 (Spherical)
  Number of Parameters (k): 3
  Predicted Covariance Matrix (Sigma_hat):
[[1.4180233 0.       ]
 [0.        1.4180233]]
  Maximum Log Likelihood: -478.0711
  BIC score: 971.1742

Model: M2 (Diagonal)
  Number of Parameters (k): 4
  Predicted Covariance Matrix (Sigma_hat):
[[0.95090939 0.        ]
 [0.         1.8851372 ]]
  Maximum Log Likelihood: -469.4564
  BIC score: 958.9553

Model: M3 (Full)
  Number of Parameters (k): 5
  Predicted Covariance Matrix (Sigma_hat):
[[0.95090939 0.62419829]
 [0.62419829 1.8851372 ]]
  Maximum Log Likelihood: -451.0760
  BIC score: 927.2051

--- Final Model Selection (Lowest BIC) ---
Selected Model: M3 (Full)
Selected Model's BIC: 927.2051
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advan